In [ ]:
import cv2
import torch
import pandas as pd

In [ ]:
path = 'model_weights.pt'
model = torch.hub.load('ultralytics/yolov5', 'custom', path, force_reload = True)

In [ ]:
# Main (Code executed in Google Colab): Writing the Processed Frames to an Output file
cap = cv2.VideoCapture("/content/drive/MyDrive/Assignment1/Test Videos/4.mp4")

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter('./Output_4.mp4', fourcc, fps, (width, height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    with torch.no_grad():
        detections = model(frame).xyxy[0].cpu().numpy()

    bb_list = []
    for row in detections:
        x1, y1, x2, y2, _, _ = row
        bb_list.append([x1, y1, x2, y2])

    if bb_list:
        for area in bb_list:
            mx1, my1, mx2, my2 = map(int, area)
            cv2.rectangle(frame, (mx1, my1), (mx2, my2), (255, 0, 0), 2)

    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

In [ ]:
# Main (Code executed in my PC): Real-Time Detection (Some frames are skipped to improve FPS performance)
cap = cv2.VideoCapture("Test Videos/4.mp4")
fcount = 0

while True:
    ret, frame = cap.read()
    if not ret: 
        break
    fcount += 1
    if fcount%7 != 0: 
        continue

    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    detections = pd.DataFrame(model(frame).xyxy[0])
    
    bb_list = []
    for _, row in detections.iterrows():
        x1, y1, x2, y2, _, _ = row
        bb_list.append([x1, y1, x2, y2])

    if bb_list:
        for area in bb_list:
            mx1, my1, mx2, my2 = map(int, area)
            cv2.rectangle(frame, (mx1, my1), (mx2, my2), (255, 0, 0), 2)
                                
    frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    cv2.imshow("Frame", frame)
    
    if cv2.waitKey(1) & 0xFF == 27: 
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)